<a href="https://colab.research.google.com/github/adalves-ufabc/2025.Q3-PLN/blob/main/2025_Q3_PLN_AULA_18_Notebook_38.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

## **Agentes e SQL [LangChain]**
---



#### **LangChain**

In [ ]:
#@title Instalando o pacote LangChain

!pip install -qU langchain

In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

1.0.5


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
#@title Definindo a chave da API da OpenAI
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


#### **Exemplo**

Neste exemplo, você aprenderá como criar um agente capaz de responder a perguntas sobre um banco de dados SQL usando agentes **LangChain**. Em linhas gerais, o agente irá:


* Obter as tabelas e esquemas disponíveis no banco de dados.

* Decidir quais tabelas são relevantes para a pergunta.

* Obter os esquemas das tabelas relevantes.

* Gerar uma consulta com base na pergunta e nas informações dos esquemas.

* Verificar a consulta em busca de erros comuns usando um LLM.

* Executar a consulta e retornar os resultados.

* Corrigir os erros detectados pelo mecanismo do banco de dados até que a consulta seja bem-sucedida.

* Formular uma resposta com base nos resultados.

In [ ]:
!pip install -qU langchain-community

Defina um modelo:

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model="gpt-4.1", api_key=OPENAI_API_KEY)

Você criará um banco de dados **SQLite**. O **SQLite** é um banco de dados leve, fácil de configurar e usar. Carregaremos o banco de dados `Chinook`, um banco de dados de exemplo que representa uma loja de mídia digital. Para sua conveniência, hospedamos o banco de dados (`Chinook.db`) em um bucket público do Google Cloud Storage.

In [ ]:
import requests, pathlib

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
local_path = pathlib.Path("Chinook.db")

if local_path.exists():
    print(f"{local_path} already exists, skipping download.")
else:
    response = requests.get(url)
    if response.status_code == 200:
        local_path.write_bytes(response.content)
        print(f"File downloaded and saved as {local_path}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as Chinook.db


Usaremos um prático `wrapper` de banco de dados SQL disponível no pacote `langchain_community` para interagir com o banco de dados. O `wrapper` fornece uma interface simples para executar consultas SQL e obter resultados:

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Artist LIMIT 5;")}')

Dialect: sqlite
Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Sample output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=modelo)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



Use `create_agent` para criar um agente com o mínimo de código. O agente interpretará a solicitação e gerará um comando SQL, que será executado pelas ferramentas. Se o comando apresentar um erro, a mensagem de erro será retornada ao modelo. O modelo poderá então examinar a solicitação original e a nova mensagem de erro e gerar um novo comando. Isso pode continuar até que o LLM gere o comando com sucesso ou atinja uma contagem regressiva. Esse padrão de fornecer feedback ao modelo — mensagens de erro, neste caso — é muito poderoso.


Inicialize o agente com um prompt descritivo do sistema para personalizar seu comportamento.

In [ ]:
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

Agora, crie um agente com o modelo, as ferramentas e o prompt:

In [ ]:
from langchain.agents import create_agent

agente = create_agent(
    modelo,
    tools,
    system_prompt=system_prompt,
)

Execute o agente em uma consulta de exemplo e observe seu comportamento:

In [ ]:
question = "Which genre on average has the longest tracks?"

for step in agente.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which genre on average has the longest tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_q6xcXpUftxdzAouQZecsVX3F)
 Call ID: call_q6xcXpUftxdzAouQZecsVX3F
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_CNHNXRUnhoQCFMn6QuLgQrNQ)
 Call ID: call_CNHNXRUnhoQCFMn6QuLgQrNQ
  Args:
    table_names: Genre, Track
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Genre" (
	"GenreId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("GenreId")
)

/*
3 rows from Genre table:
GenreId	N

**Referência:**

> https://docs.langchain.com/oss/python/langchain/sql-agent